In [26]:
import pandas as pd
import networkx as nx
from tqdm import tqdm
import os
import glob

In [12]:
def create_all_edges(parents, children):
    res = []
    
    for parent in parents:
        for child in children:
            res.append((parent, child))

    return res

def generator(G):
    while not nx.is_directed_acyclic_graph(G):
        cycle = nx.find_cycle(G)
        yield cycle[0]

def make_clean_graph(in_path, out_path):
    df = pd.read_csv(in_path,index_col=0)

   # df2 = df1[~df1['Definition'].isna()] # do not use it anymore
    G = nx.from_pandas_edgelist(df, 'Hypernym',
                                 'Synset', create_using=nx.DiGraph)

    nodes_to_add = []
    edges_to_add = []
    nodes_to_delete = []

    for node in G:
        if len(str(node).split('\t')) > 1:
            nodes_to_add.extend((str(node).split('\t')))
            edges_to_add.extend(create_all_edges(str(node).split('\t'), list(G[node])))
            nodes_to_delete.append(node)

    G.add_nodes_from(nodes_to_add)
    G.add_edges_from(edges_to_add)
    G.remove_nodes_from(nodes_to_delete)

    for bad_edge in tqdm(generator(G)):
        G.remove_edge(*bad_edge)

    nx.write_edgelist(G, out_path, delimiter='\t')

In [28]:
all_data = glob.glob(os.path.join("../data/omw/", "*.csv"))
for path in tqdm(all_data):
    lang = path.split("/")[-1].replace(".csv", "")
    save_path = '../data/omw/{}_isa.edgelist'.format(lang)

    make_clean_graph(path, save_path)

7it [00:01,  5.73it/s]00:00<?, ?it/s]
 20%|██        | 1/5 [00:01<00:06,  1.62s/it]

In [16]:
G = nx.read_edgelist(out_path, delimiter="\t", create_using=nx.DiGraph)

In [22]:
G.number_of_nodes()

31768